<a href="https://colab.research.google.com/github/cmagliano/Proj/blob/main/exemplo_ragbasic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Configuração do Ambiente: Instalamos as bibliotecas necessárias para trabalhar com embeddings (sentence-transformers), busca eficiente (faiss) e a API da OpenAI.

Preparação dos Dados: Criamos uma lista simples de documentos para nosso exemplo.

Modelo de Recuperação:

Usamos sentence-transformers para criar embeddings dos documentos.
Usamos faiss para indexar esses embeddings e permitir a busca eficiente.
Modelo de Geração:

Configuramos a chave da API da OpenAI.
Criamos uma função retrieve_and_generate que recebe uma consulta, recupera documentos relevantes, concatena esses documentos como contexto e usa a API da OpenAI para gerar uma resposta.
Chatbot:

A função retrieve_and_generate pode ser chamada a partir de uma interface de chatbot para interagir com os usuários.

1. Configuração do Ambiente
Instale as bibliotecas necessárias:

In [3]:
!pip install transformers
!pip install faiss-cpu
!pip install sentence-transformers
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.0 MB/s eta 0:00:00


2. Preparação dos Dados
Crie um conjunto de documentos de exemplo:

In [9]:
documents = [
    "The Eiffel Tower is located in Paris.",
    "The Great Wall of China is visible from space.",
    "The Mona Lisa was painted by Leonardo da Vinci.",
    "The capital of Japan is Tokyo.",
    "The Pyramids of Giza are one of the Seven Wonders of the Ancient World. The Pyramids are located in Egypt"
    "The Amazon River is in north of Brazil and it is the longest one in the world."
]


3. Modelo de Recuperação
Usaremos sentence-transformers para criar embeddings dos documentos e faiss para realizar a busca eficiente:

In [10]:
from sentence_transformers import SentenceTransformer
import faiss

# Carregar modelo para embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Criar embeddings para os documentos
document_embeddings = model.encode(documents)

# Indexar embeddings usando FAISS
index = faiss.IndexFlatL2(document_embeddings.shape[1])
index.add(document_embeddings)


4. Modelo de Geração
Integraremos o modelo de geração da OpenAI:

In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Carregar tokenizer e modelo de geração
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model_gen = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

def retrieve_and_generate(query, top_k=2):
    # Codificar a consulta
    query_embedding = model.encode([query])

    # Recuperar documentos relevantes
    distances, indices = index.search(query_embedding, top_k)
    retrieved_docs = [documents[i] for i in indices[0]]

    # Concatenar documentos recuperados
    context = " ".join(retrieved_docs)

    # Gerar resposta usando o contexto
    inputs = tokenizer.encode("question: " + query + " context: " + context, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model_gen.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)




In [7]:
# Exemplo de consulta
query = "Where is the Eiffel Tower located?"
response = retrieve_and_generate(query)
print(response)

question: Where is the Eiffel Tower located? context: The Eiffle Tower is located in Paris. The Great Wall of China is visible from space. It is located on the east coast of China.


In [8]:
# Exemplo de consulta 2
query = "Where is the Great Pyramid located?"
response = retrieve_and_generate(query)
print(response)

The Pyramids of Giza are one of the Seven Wonders of the Ancient World. The Pyramids are located in Egypt. The Great Wall of China is visible from space. The Sphinx is located in Saudi Arabia.


In [13]:
# Exemplo de consulta 2
query = "Where is the longest river in the world?"
response = retrieve_and_generate(query)
print(response)

The Amazon River is in north of Brazil and it is the longest one in the world. The Great Wall of China is visible from space. The Pyramids of Giza are one of the Seven Wonders of the Ancient World.
